In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [3]:
df_panel1 = pd.read_csv(os.path.join('DATA', 'df_panel1.csv')) 

In [10]:
df_panel1.columns

Index(['Unnamed: 0', 'gvkey_hscode6', 'date', 'vol', 'code2', 'month',
       'quarter', 't', '1', '2', '3', '4', 'vol_log', 't_log', 'vol_log_hat',
       'R', 'sigmaR', 'source1', 'N', 'T', 'despues', 'antes', 'consistency_8',
       'consistency_china', 'consistency_no_china', 'gvkey', 'hscode6'],
      dtype='object')

In [11]:
df = df_panel1[df_panel1['consistency_8']==1]
df = pd.DataFrame(df[['gvkey_hscode6', 'source1']].groupby('gvkey_hscode6')['source1'].nunique())
df.reset_index(inplace=True)
df = df[df['source1']==2]
df

,gvkey_hscode6,source1
5,10005_853921,2
13,100095_611610,2
27,10016_854442,2
37,100609_850131,2
42,100609_853890,2
...,...,...
3777,8562_870829,2
3783,8823_940360,2
3797,9016_850300,2
3855,9818_851830,2


In [12]:
gvkeys = list(df['gvkey_hscode6'])
gvkeys

['10005_853921',
 '100095_611610',
 '10016_854442',
 '100609_850131',
 '100609_853890',
 '100609_854430',
 '100609_870829',
 '100609_870830',
 '10115_850110',
 '10115_940290',
 '101202_630790',
 '10198_620342',
 '10198_620520',
 '102523_851629',
 '102523_854430',
 '10275_853210',
 '10332_851770',
 '10332_950300',
 '10507_610711',
 '10519_610910',
 '10530_630260',
 '10530_854442',
 '105936_620520',
 '10622_852580',
 '10622_853929',
 '10983_853669',
 '11060_611595',
 '11060_640340',
 '11060_640391',
 '11191_853225',
 '11259_851822',
 '11259_851830',
 '116004_620462',
 '11672_940520',
 '118122_853400',
 '118122_854430',
 '11818_950699',
 '118574_851830',
 '120444_854449',
 '12124_591190',
 '12141_620520',
 '12389_620520',
 '12389_620630',
 '125014_854420',
 '126721_852580',
 '12688_853921',
 '12788_940510',
 '12868_620342',
 '1300_640391',
 '133367_850750',
 '13505_850650',
 '13570_851680',
 '135965_851762',
 '13709_940490',
 '13906_630720',
 '13906_850440',
 '140033_851762',
 '14062_8531

In [14]:
df = df_panel1[df_panel1['gvkey_hscode6']==gvkeys[0]][['date', 'gvkey_hscode6','source1', 'vol_log', 'vol_log_hat', 'sigmaR']]
df.to_csv(os.path.join('DATA', 'plot_data.csv'))

In [4]:
def get_N():
    df = df_panel1[df_panel1['consistency_8']==1][['gvkey_hscode6', 'source1']].groupby(['gvkey_hscode6']).nunique()
    df.reset_index(inplace=True)
    df.rename(columns={'source1': 'N'}, inplace=True)
    dictio = {gvhs: n for gvhs, n in zip(df['gvkey_hscode6'], df['N'])}
    
    N = dict()
    N[1] = list(df['gvkey_hscode6'][df['N']==1].unique())
    N[2] = list(df['gvkey_hscode6'][df['N']==2].unique())
    N[3] = list(df['gvkey_hscode6'][df['N']==3].unique())
    N[0] = list(set(df_panel1['gvkey_hscode6'].unique()).difference(set(N[1])).difference(set(N[2])).difference(set(N[3])))

    return N

In [5]:
N = get_N()

In [23]:
df = pd.DataFrame(df_panel1[['gvkey_hscode6', 'source1']].groupby('gvkey_hscode6')['source1'].nunique())
df.reset_index(inplace=True)
df_sources = df.copy()
df_sources

,gvkey_hscode6,source1
0,10000_850440,5
1,10000_850490,3
2,10000_853641,1
3,10000_870899,7
4,10002_620342,3
...,...,...
5060,9850_852580,2
5061,9899_851762,6
5062,9899_852871,8
5063,9899_852910,9


In [27]:
lista = []
for gvkey in N[2]:
    df = df_panel1[df_panel1['gvkey_hscode6']==gvkey].copy()
    sources = list(df[df['consistency_8']==1]['source1'].unique())

    if len(sources)==1: continue
    if sources[0]=='China': lista.append(gvkey)

In [28]:
len(lista)

107

In [29]:
df_panel1['id_china'] = 0

In [32]:
for gvkey in lista:
    dates = df_panel1[df_panel1['gvkey_hscode6']==gvkey][np.logical_and(df_panel1['consistency_8']==1, df_panel1['source1']=='China')]['date']
    df_panel1.loc[np.logical_and(df_panel1['gvkey_hscode6']==gvkey, df_panel1['date'].isin(dates)), 'id_china'] = 1

In [33]:
df_panel1['id_china'].unique()

array([0, 1], dtype=int64)

In [37]:
df_panel1['1st_consistency'] = 0
df_panel1['2nd_consistency'] = 0

In [38]:
for gvkey in N[2]:
    df = df_panel1[df_panel1['gvkey_hscode6']==gvkey].copy()
    sources = list(df[df['consistency_8']==1]['source1'].unique())

    if len(sources)==1: continue
    
    dates = df_panel1[df_panel1['gvkey_hscode6']==gvkey][np.logical_and(df_panel1['consistency_8']==1, df_panel1['source1']==sources[0])]['date']
    df_panel1.loc[np.logical_and(df_panel1['gvkey_hscode6']==gvkey, df_panel1['date'].isin(dates)), '1st_consistency'] = 1
    
    dates = df_panel1[df_panel1['gvkey_hscode6']==gvkey][np.logical_and(df_panel1['consistency_8']==1, df_panel1['source1']==sources[1])]['date']
    df_panel1.loc[np.logical_and(df_panel1['gvkey_hscode6']==gvkey, df_panel1['date'].isin(dates)), '2nd_consistency'] = 1
    

In [39]:
df_panel1['T1'].unique()

array([0, 1], dtype=int64)

In [40]:
df_panel1['T2'].unique()

array([0, 1], dtype=int64)

In [41]:
df_panel1.to_csv(os.path.join('DATA', 'df_panel1_.csv'))

In [42]:
df_panel2 = df_panel1[df_panel1['gvkey_hscode6'].isin(N[2])].copy()
df_panel2 = df_panel2[df_panel2["consistency_8"]==1]
df_panel2

,Unnamed: 0,gvkey_hscode6,date,vol,code2,month,quarter,t,1,2,...,despues,antes,consistency_8,consistency_china,consistency_no_china,gvkey,hscode6,id_china,1st_consistency,2nd_consistency
221,221,10005_853921,2010-04-01,1.69,85,4,2,5,0,1,...,8,1,1,1,0,10005,853921,1,1,0
222,222,10005_853921,2010-07-01,0.41,85,7,3,6,0,0,...,7,2,1,1,0,10005,853921,1,1,0
223,223,10005_853921,2010-10-01,0.24,85,10,4,7,0,0,...,6,3,1,1,0,10005,853921,1,1,0
224,224,10005_853921,2011-01-01,1.86,85,1,1,8,1,0,...,5,4,1,1,0,10005,853921,1,1,0
225,225,10005_853921,2011-04-01,2.54,85,4,2,9,0,1,...,4,5,1,1,0,10005,853921,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188393,188393,9899_854442,2018-07-01,5.61,85,7,3,38,0,0,...,5,9,1,1,0,9899,854442,0,0,1
188394,188394,9899_854442,2018-10-01,20.84,85,10,4,39,0,0,...,4,10,1,1,0,9899,854442,0,0,1
188395,188395,9899_854442,2019-01-01,11.16,85,1,1,40,1,0,...,3,11,1,1,0,9899,854442,0,0,1
188396,188396,9899_854442,2019-04-01,4.28,85,4,2,41,0,1,...,2,12,1,1,0,9899,854442,0,0,1


In [43]:
df_panel2.to_csv(os.path.join('DATA', 'df_panel2_.csv'))